In [ ]:
# Minimal DR Triple Cross-Fit
# Based on Ke et al. (2023)

!pip install opendatasets xgboost

import pandas as pd
import numpy as np
import opendatasets as od
import os, re, cv2, gc
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# 1. DATA LOADING
od.download("https://www.kaggle.com/datasets/khanfashee/nih-chest-x-ray-14-224x224-resized/data")

CSV_PATH = '/content/nih-chest-x-ray-14-224x224-resized/Data_Entry_2017.csv'
IMG_DIR = '/content/nih-chest-x-ray-14-224x224-resized/images-224/images-224'
TARGET_N = 3500       # target sample size
IMG_SIZE = (224,224)
RANDOM_SEED = 42

# Metadata preprocessing
df = pd.read_csv(CSV_PATH)
df['Finding Labels'] = df['Finding Labels'].astype(str).str.strip()

def list_all_labels(series):
    labels = set()
    for row in series:
        for lab in row.split('|'):
            labels.add(lab.strip())
    return sorted(labels)

def has_label(s, lab):
    lab_esc = re.escape(lab)
    return bool(re.search(rf'(^|\|){lab_esc}($|\|)', s))

all_labels = list_all_labels(df['Finding Labels'])
for lab in all_labels:
    df[lab] = df['Finding Labels'].apply(lambda s: 1 if has_label(s, lab) else 0)

df['Age_clean'] = df['Patient Age'].astype(str).str.extract(r'(\d+)').astype(float)
df['image_path'] = df['Image Index'].apply(lambda f: os.path.join(IMG_DIR, f))
df = df[df['image_path'].apply(os.path.exists)].reset_index(drop=True)


# NEW SAMPLING: All pneumonia cases + random non-pneumonia until 3500
np.random.seed(RANDOM_SEED)

# All pneumonia-positive cases
pneu_pos = df[df['Pneumonia'] == 1]

# All non-pneumonia cases
pneu_neg = df[df['Pneumonia'] == 0]

# How many negatives needed to reach target
needed_neg = max(0, TARGET_N - len(pneu_pos))

# Randomly sample negatives
pneu_neg_sample = pneu_neg.sample(n=needed_neg, random_state=RANDOM_SEED)

# Combine positives + sampled negatives
sample_df = pd.concat([pneu_pos, pneu_neg_sample], axis=0)

# Shuffle dataset
sample_df = sample_df.sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)

print("Final sample size:", len(sample_df))
print("Pneumonia cases:", sample_df['Pneumonia'].sum())
print("Non-Pneumonia cases:", len(sample_df) - sample_df['Pneumonia'].sum())


# Covariates (Z), Treatment (T), Outcome (Y)
covar_disease_cols = [c for c in all_labels if c not in ['Pneumonia','Infiltration','No Finding']]
gender_map = {'M': 1, 'F': 0}
sample_df['Gender_enc'] = sample_df['Patient Gender'].map(gender_map)
view_dummies = pd.get_dummies(sample_df['View Position'], prefix="ViewPos")

Z = np.column_stack([
    sample_df['Age_clean'].to_numpy().astype(float),
    sample_df['Gender_enc'].to_numpy().astype(int),
    sample_df[covar_disease_cols].to_numpy().astype(int),
    view_dummies.to_numpy().astype(int)
])
Y = sample_df['Pneumonia'].to_numpy().astype(int)
T = sample_df['Infiltration'].to_numpy().astype(int)

print("Z:", Z.shape, "Y:", Y.shape, "T:", T.shape)



# 2. IMAGE PREPROCESSING
def preprocess_image_rgb(path, size=IMG_SIZE, dtype=np.float32):
    img = cv2.imread(path)
    if img is None: return None
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, size)
    return (img.astype(dtype) / 255.0)

X_list, drop_idx = [], []
for i, p in enumerate(sample_df['image_path'].tolist()):
    arr = preprocess_image_rgb(p)
    if arr is None: drop_idx.append(i)
    else: X_list.append(arr)

if drop_idx:
    keep = np.ones(len(sample_df), dtype=bool)
    keep[drop_idx] = False
    sample_df = sample_df[keep].reset_index(drop=True)
    Z, Y, T = Z[keep], Y[keep], T[keep]

X = np.stack(X_list, axis=0).astype(np.float32)
del X_list; gc.collect()

print("X:", X.shape, "Z:", Z.shape, "T:", T.shape, "Y:", Y.shape)


# 3. HDFPCA
def perform_hdfpca(X, variance_threshold=0.95):
    n = X.shape[0]
    mean_image = np.mean(X, axis=0)
    X_centered = X - mean_image
    X_mat = X_centered.reshape(n, -1)
    U, s, Vt = np.linalg.svd(X_mat.T, full_matrices=False)
    eigenvalues = s**2 / (n - 1)
    cumulative_var = np.cumsum(eigenvalues) / np.sum(eigenvalues)
    n_components = np.argmax(cumulative_var >= variance_threshold) + 1
    U_selected = U[:, :n_components]
    s_selected = s[:n_components]
    eigenscores = (np.diag(s_selected) @ Vt[:n_components, :]).T
    return eigenscores, U_selected, s_selected, mean_image

def transform_with_hdfpca(X, U, s, mean_image):
    n = X.shape[0]
    X_centered = X - mean_image
    X_mat = X_centered.reshape(n, -1)
    return X_mat @ U


# 4. DR TRIPLE CROSS-FIT (Logistic Regression only)
def dr_score(y, t, ps, m1, m0):
    ps = np.clip(ps, 1e-6, 1-1e-6)
    return (m1 - m0) + (t * (y - m1) / ps) - ((1 - t) * (y - m0) / (1 - ps))

def triple_crossfit_dr_simple(X_imgs, Z, T, Y, cpv=0.95, P=3, random_state=42):
    rng = np.random.RandomState(random_state)
    n = len(Y)
    ate_list = []

    for p in range(P):
        idx = np.arange(n)
        rng.shuffle(idx)
        folds = np.array_split(idx, 4)
        rot_ates = []

        for r in range(4):
            A,B,C,D = folds[r], folds[(r+1)%4], folds[(r+2)%4], folds[(r+3)%4]

            # HDFPCA on fold A
            try:
                eig_A,U_A,s_A,mean_A = perform_hdfpca(X_imgs[A], variance_threshold=cpv)
            except: continue

            # Transform folds
            ES_B = transform_with_hdfpca(X_imgs[B], U_A, s_A, mean_A)
            ES_C = transform_with_hdfpca(X_imgs[C], U_A, s_A, mean_A)
            ES_D = transform_with_hdfpca(X_imgs[D], U_A, s_A, mean_A)

            XB = np.column_stack([Z[B], ES_B])
            XC = np.column_stack([Z[C], ES_C])
            XD = np.column_stack([Z[D], ES_D])

            scaler = StandardScaler().fit(np.vstack([XB,XC]))
            XB,XC,XD = scaler.transform(XB),scaler.transform(XC),scaler.transform(XD)

            # Propensity model on B
            try:
                prop_model = LogisticRegression(max_iter=1000).fit(XB, T[B])
            except: continue

            #  Outcome models on C
            XC_t1,Y_t1 = XC[T[C]==1], Y[C][T[C]==1]
            XC_t0,Y_t0 = XC[T[C]==0], Y[C][T[C]==0]
            if len(XC_t1)==0 or len(XC_t0)==0: continue

            try:
                out1_model = LogisticRegression(max_iter=1000).fit(XC_t1, Y_t1)
                out0_model = LogisticRegression(max_iter=1000).fit(XC_t0, Y_t0)
            except: continue

            # Predict on D
            try:
                ps_D = prop_model.predict_proba(XD)[:,1]
                m1_D = out1_model.predict_proba(XD)[:,1]
                m0_D = out0_model.predict_proba(XD)[:,1]
            except: continue

            phi = dr_score(Y[D], T[D], ps_D, m1_D, m0_D)
            rot_ates.append(phi.mean())

        if rot_ates:
            ate_list.append(np.mean(rot_ates))

    if not ate_list:
        return {"ATE": np.nan,"SE":np.nan,"CI":(np.nan,np.nan),"P_used":0}

    ate_hat = np.median(ate_list)
    se_hat = np.std(ate_list)/np.sqrt(len(ate_list))
    ci_low, ci_high = ate_hat - 1.96*se_hat, ate_hat + 1.96*se_hat

    return {"ATE": ate_hat,"SE":se_hat,"CI":(ci_low,ci_high),"P_used":len(ate_list)}


# 5. RUN
print("Running DR Triple Cross-Fit (LogReg only)...")
result = triple_crossfit_dr_simple(X, Z, T, Y, cpv=0.95, P=3, random_state=42)

print("\n===== RESULTS =====")
print(f"ATE: {result['ATE']:.4f}")
print(f"SE : {result['SE']:.4f}")
print(f"95% CI: [{result['CI'][0]:.4f}, {result['CI'][1]:.4f}]")
print(f"Partitions used: {result['P_used']}")